Панасюк Настя, бкл182

###### 0. Импорты

In [1]:
import re
from pymystem3 import Mystem
from pymorphy2 import MorphAnalyzer
from nltk.tokenize import word_tokenize, sent_tokenize, pos_tag
from natasha import Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, Doc
import pandas as pd
from sklearn.metrics import accuracy_score
import spacy
from nltk.corpus import webtext
import en_core_web_sm
from flair.data import Sentence
from flair.models import SequenceTagger
import json

In [297]:
m = Mystem()
morph = MorphAnalyzer()
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)

###### 1. Русский

In [526]:
with open('ru.txt', encoding='utf-8') as f:
    rutext = f.read()
ruwords =  [w.strip(' .,?“”').split('=') for w in rutext.replace('  ', ' ').split(' ')]  # разбиваю текст на множество токенов с разметкой
rupassage = re.sub('[A-z=]', '', rutext)  # создаю абзац без разметки

In [528]:
print(rutext)

Иванов=NOUN проснулся=V печальным=A. Дома=ADV “иваново=A детство=NOUN” никто=NPRO не=PART смотрел=V. Что=NPRO ищет=V он=NPRO в=PR стране=NOUN далекой=A? Что-то=ADVPRO Резвых=NOUN не=PART едет=V домой=ADV. Мороженое=A мясо=NOUN оставил=V в=PR холодильнике=NOUN. А=CONJ мороженое=NOUN забыл=V на=PR столе=NOUN, и=CONJ оно=NPRO растеклось=V. Гэндзи-моногатари=NOUN совсем=ADV не=PART читабельный=A роман=NOUN. На=PR берегу=NOUN безымянной=A реки=NOUN берегу=V свою=APRO красоту=NOUN. На=PR берегу=NOUN берегу=V лист=NOUN. Посмотри=V в=PR НКРЯ=NOUN. Очень=ADV по-дурацки=ADV, по-глупому=ADV светила=V луна=NOUN. В=PR Лас-вегасе=NOUN Ганеша=NOUN купил=V 15=NUM летних=A брюк=NOUN. 15-летние=A Юны=NOUN шлют=V привет=NOUN Намам=NOUN. Детская=A пила=NOUN пила=V морс=NOUN. Фрукт=NOUN был=V спел=A, спел=V калинку=NOUN. Попса=NOUN попсе=NOUN рознь=NOUN, лайн-ап=NOUN у=PR “Боли”=NOUN дурацкий=A. КГБшник=NOUN Петров=NOUN из=PR города=NOUN уехал=V. Петров=A город=NOUN переводился=V с=PR острова=NOUN. Облако=

###### 1.1. Почему я считаю текст сложным?

Коротко:<br>
<li>фамилии чередуются с генитивами (Иванов - иваново)</li>
<li>омографы: мою ванную хлоркой (понятно, что мою - глагол только из модели управления), на берегу-N берегу-V <br>
<li>неизвестные слова, аббревиатуры <br>
<li>краткие формы (спел), Свят как имя (Святослав) vs "свят" краткое прилагательное <br>
<li>сложности с наречие vs существительные (чудом, вечером) <br>
остальное скорее всего довольно очевидное, но если нужно, чтобы я объяснила, я готова <a href='t.me/mjolnika'>мой тг</a>

In [502]:
# юнификация всех-всех тегов всех парсеров

correctingpos = {'APRO':'A', 'ADVPRO':'ADV', 'SPRO':'PRO', 'S':'NOUN',
               'ADVB':'ADV', 'ADJF':'A', 'VERB':'V', 'GRND':'V',
               'ADJS':'A', 'PRCL':'PART', 'NPRO':'PRO', 'PREP':'PR',
               'PROPN':'NOUN', 'ADP':'PR',  'AUX':'V', 'DET':'A',
               'ADJ': 'A', 'CCONJ':'CONJ', 'PRON':'PRO', 'SCONJ':'CONJ'}

In [451]:
# функция, которая переводит любые теги к нужному виду

def correcttogold(parsed, correctingpos):
    for item in parsed:
        if item[1] in correctingpos.keys():
            tag = item[1]
            item[1] = correctingpos[tag]
    return parsed

In [529]:
ru_cor = correcttogold(ruwords, correctingpos)  # скорректированные теги из моей "экспертной" разметки

###### 1.2 mystem

In [531]:
analysis = m.analyze(rupassage)  # запуск
mystempos = []

# парс выдачи
for item in analysis:
    if item.get('text', []).isalpha() or item.get('text', []).isdigit() or ('-' in item.get('text') and len(item)>1):
        if item.get('analysis', []):
            gr = item.get('analysis', [])[0]['gr'].replace('=', ',')
            pos, _ = gr.split(',', 1)
            mystempos.append([item.get('text'), pos])
        elif item.get('text').isdigit() or item.get('text').isalpha():  # возвращаю разделенные дефисами цифры и слова
            pos='None'
            mystempos.append([item.get('text'), pos])

mystem_cor = correcttogold(mystempos, correctingpos)  # коррекция

Вывод убираю, потому что много. <br>
<ol><li>Мне не нравится, что майстем делит по дефису, это неудобно для сравнения. В итоге вылезает какое-то количество доп. токенов, и изначально неочевидно, где вылезет. (майстем дробил "лайн-ап", потому не мог разобрать "ап". В итоге, чтобы не потерять информацию, приходится прописывать много исключений. Уж лучше бы не дробил или хотя бы проверял, есть ли значение у частей, на которые дробт (типа "приходите в белой-черной одежде" подробить норм, но "бело-черной" уже немного другое значение)</li><li>Еще он не выделяет числа (15), что тоже приводит к потере смысла, типа, 15-летние молодые люди != летние молодые люди.</li></ol>

###### 1.3 pymorphy

In [533]:
tokens = [w for w in word_tokenize(rupassage) if w.isalpha() or w.isdigit() or '-' in w]
parsed = [morph.parse(token)[0].tag.POS for token in tokens]

pymorphypos = []
for i, j in zip(tokens, parsed):
    pymorphypos.append([i, j])
    
pym_cor = correcttogold(pymorphypos, correctingpos)

###### 1.4 natasha

In [538]:
doc = Doc(rupassage)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doctoken = doc.tokens

natashapos = []
for item in doctoken:
    if item.pos!='PUNCT':
        natashapos.append([item.text, item.pos])
        
natasha_cor = correcttogold(natashapos, correctingpos)

##### 1.5. Делаю большой словарь со всем-всем-всем, чтобы потом наглядно через датафрэйм посмотреть теги еще и глазами + вылечить ошибки деления майстема и его "расползновение"

In [543]:
alldict = []
error = []  # сюда записываю "огрызки", которые придется выбросить
for i, parselist in enumerate(zip(ru_cor, mystem_cor, pym_cor, natasha_cor)):
    
    gold, my, py, nat = parselist
    word = gold[0]
    if mystem_cor[i+len(error)][0]==word or word.startswith(mystem_cor[i+len(error)][0]):  # делаю поправку на "огрызки"
        myst=mystem_cor[i+len(error)][1]
    else:
        error.append(mystem_cor[i+len(error)])
        myst=mystem_cor[i+len(error)][1]
    token = {'word':word, 'gold':gold[1], 'mystem':myst, 'pymorphy':str(py[1]), 'natasha':nat[1]}

    alldict.append(token)

In [517]:
error  # что выкинулось

[['моногатари', 'NOUN'], ['летние', 'A'], ['ап', 'None']]

###### 1.6. Датафрэйм для наглядности

In [544]:
df = pd.DataFrame.from_dict(alldict)
pd.set_option('display.max_rows', 130)
display(df)

In [545]:
pd.set_option('display.max_rows', 130)

In [546]:
display(df)

,word,gold,mystem,pymorphy,natasha
0,Иванов,NOUN,NOUN,NOUN,NOUN
1,проснулся,V,V,V,V
2,печальным,A,A,A,A
3,Дома,ADV,ADV,NOUN,NOUN
4,иваново,A,A,NOUN,A
5,детство,NOUN,NOUN,NOUN,NOUN
6,никто,PRO,PRO,PRO,PRO
7,не,PART,PART,PART,PART
8,смотрел,V,V,V,V
9,Что,PRO,PRO,CONJ,CONJ


In [547]:
gold = df.gold.to_list()  # конвертирую для подсчета точности
mystem = df.mystem.to_list()
pymorphy = df.pymorphy.to_list()
natasha = df.natasha.to_list()

###### 1.7. Считаю точность разборов

In [174]:
def accuracy(gold, parser):
    print("Accuracy: %.4f" % accuracy_score(parser, gold))

In [548]:
my_acc = accuracy(gold, mystem)

Accuracy: 0.8537


In [549]:
py_acc = accuracy(gold, pymorphy)

Accuracy: 0.8537


In [550]:
na_acc = accuracy(gold, natasha)

Accuracy: 0.8455


У майстема и пайморфи получилась одинаковая точность, у наташи на один верный ответ меньше (проверено, как там высчитывается), поэтому использовать я буду все равно наташу. В моем тестовом тексте было больше производных наречий (вечером, чудом), которые наташа не опознала. Но с точки зрения распознавания остального меня она больше устраивает.<br> В паре мест записала формы глаголов/причастий в существительные, но это не так печально.

###### 2. Английский

In [226]:
with open('en.txt', encoding='utf-8') as f:
    entext = f.read()

In [227]:
entext

'Heat=VB water=NN in=IN a=DT large=JJ vessel=NN. Flies=NNS like=VBZ a=DT flower=NN. Each=DT one=CD plant=VB one=CD. He=PRP will=MD race=VB the=DT car=NN. 15=CD boats=NNS floated=VBN down=RP the=DT river=NN sank=VBD. I=PRP wanted=VBD to=TO win=VB a=DT Subaru=NNP WRX=NNP. I=PRP might=MD not=RB reply=VB. The=DT sailor=NN dogs=VBZ the=DT hatch=NN. Dr=NNP Young=NNP was=VBD not=RB anymore=RB young=JJ. A=DT cat=NN walks=VBZ on=IN the=DT catwalk=NN. How=WRB white=JJ everything=NN is=VBZ, how=WRB quiet=JJ, how=WRB snowed-in=JJ. I=PRP am=VBP learning=VBG peacefulness=NN, lying=VBG by=IN myself=PRP quietly=RB and=CC not=RB lying=VBG myself=PRP anymore=RB. Can=MD you=PRP make=VB "Can"=NNP come=VB back=IN once=RB more=RBR? Daveed=NNP Diggs=NNP diggs=VBZ this=DT music=NN, man=NN. Still=None life=NN continued=VBD to=TO develop=VB in=IN Flemish=JJ painting=NN after=RB the=DT separation=NN of=IN the=DT North=NNP and=CC South=NNP, but=CC is=VBZ rare=JJ in=IN Dutch=JJ painting=NN. PTA=NNP blows=VBZ up=RP

###### 2.1. Текст сложный из-за омографов

+ есть необычное слово Still life, которое вообще-то одно существительное. Но никто из парсеров не разобрал. <br>
+ Глаголы совпадают с существительными, конверсия итд.<br>
+ Имена собственные, которые совпадают с какими-то формами глаголов, прилагательные, от имен собственных. Парсеры разбирают по-разному.<br><br>Еще я сделала "золотую" разметку по стандартам английской, чтобы не переприводить к моему стандарту потом.

In [228]:
enwords = re.findall('[A-z0-9-\" ]+=[A-z$]+', entext)  # вычленяю токены с разметкой

In [229]:
enwords = [w.strip(' ').split('=') for w in enwords]  # получаю список токенов с разметкой

In [ ]:
enpassage = sub('=[A-z$]+', '', entext)  # чистый текст, который будет подаваться парсерам

###### 2.2. Spacy

In [207]:
# Обрабатываем текст
doc = nlp(enpassage)

In [208]:
spacy_w = []
for s in doc.sents:
    for t in s:
        if t.pos_ != 'PUNCT':
            spacy_w.append([t.text, t.tag_])

spacy побил по дефисам, потом будет решать как с майстемом

###### 2.3 Flair

In [211]:
sentences = sent_tokenize(enpassage)

In [212]:
sentences = [Sentence(sent, use_tokenizer=True) for sent in sentences]

tagger: SequenceTagger = SequenceTagger.load('pos')
tagger.predict(sentences)

2020-10-17 21:09:25,840 loading file C:\Users\mjo\.flair\models\en-pos-ontonotes-v0.5.pt


In [213]:
flairtagged = ''
for item in sentences:
    flairtagged += str(item.to_tagged_string())  # сначала собираем текст, чтобы из него потом регулярками достать теги

flair_w = re.findall('([A-z0-9-]+) <([A-z$]+?)>', flairtagged)  # получили список токенов-разборов

###### 2.4. Nltk

In [217]:
nltk_w = [item for item in pos_tag(word_tokenize(enpassage)) if item[1].isalpha() or '$' in item[1]]

###### 2.5. Датафрэйм для английского, все то же самое

In [232]:
endict = []  
enrror = []
for i, parselist in enumerate(zip(enwords, nltk_w, spacy_w, flair_w)):
    
    gold, nl, sp, fl = parselist
    
    word = gold[0].strip('"')
    if spacy_w[i+len(enrror)][0]==word or word.startswith(spacy_w[i+len(enrror)][0]):
        spcy=spacy_w[i+len(enrror)][1]
    else:
        enrror.append(spacy_w[i+len(enrror)])
        spcy=spacy_w[i+len(enrror)][1]
        
    token = {'word':word, 'gold':gold[1], 'nltk':nl[1], 'spacy':spcy, 'flair':fl[1]}
    endict.append(token)

In [233]:
edf = pd.DataFrame.from_dict(endict)
pd.set_option('display.max_rows', 120)
display(edf)

,word,gold,nltk,spacy,flair
0,Heat,VB,NNP,NN,NN
1,water,NN,NN,NN,NN
2,in,IN,IN,IN,IN
3,a,DT,DT,DT,DT
4,large,JJ,JJ,JJ,JJ
5,vessel,NN,NN,NN,NN
6,Flies,NNS,NNS,NNS,VBZ
7,like,VBZ,IN,IN,IN
8,a,DT,DT,DT,DT
9,flower,NN,NN,NN,NN


In [234]:
egold = edf.gold.to_list()
nltkk = edf.nltk.to_list()
spccy = edf.spacy.to_list()
fllair = edf.flair.to_list()

###### 2.6. Точность

In [235]:
nl_acc = accuracy(egold, nltkk)
sp_acc = accuracy(egold, spccy)
fl_acc = accuracy(egold, fllair)

Accuracy: 0.8448
Accuracy: 0.8966
Accuracy: 0.9138


Flair оказался самым точным

###### 3. Функции для улучшения сентимент анализа в русском 

Я возьму данные из предыдущей домашней работы, чтобы проверять на них гипотезы, что может получиться. Беру наташу, потому что разница в точности была незначительной.

In [239]:
with open('../hw1/first500.json', 'r', encoding='utf-8') as f:
    firstbatch = json.load(f)
with open('../hw1/500further.json', 'r', encoding='utf-8') as f:
    secondbatch = json.load(f)
reviews = firstbatch + secondbatch

In [248]:
good = []
bad = []
for item in reviews:  # разделяю отзывы по оценке 0-2 и 4.5-5
    if float(item['mark'])<3:
        bad.append(item['review'])
    elif item['mark']=='5':
        good.append(item['review'])

In [251]:
goodsample = good[:80]  #срезаю часть рецензий
badsample = bad[11:]

In [254]:
wholegood = '\n'.join(goodsample)  # буду пробовать на выборке с хорошими рецензиями

###### 3. 1. Работа с выборкой текстов

Подготовка

In [256]:
goodoc = Doc(wholegood)
goodoc.segment(segmenter)
goodoc.tag_morph(morph_tagger)
goodoc.parse_syntax(syntax_parser)

good_sents = goodoc.sents  # бью по предложениям, чтобы потом по ним итерироваться. Все через наташу.

1. Буду доставать <b>предикаты</b>, чтобы найти оценки. Часто в предикатах именно оценка.

In [401]:
#1 
for sent in good_sents:
    occurence = 0
    for item in sent.tokens:  # ищу, чтобы прилагательное было "корнем", т.е. предикатом
        if item.rel=='root' and item.pos=='ADJ':
            occurence = 1
            head_id = item.id
            head_text= item.text 
            head_gen = item.feats.get('Gender', [])
            
    if occurence ==1:  # если в предложение такое есть, итерируюсь заново и ищу существительные, которые согласованы с предикатом
        for item in sent.tokens:
            if item.head_id == head_id and (item.pos=='NOUN') and item.feats['Gender']==head_gen:
                print(item.text, head_text)
            if item.head_id == head_id and item.text.lower() =='не':  # также проверяю на отрицание, что отрицают.
                print(item.text, head_text)  # условного "книга не шикарна" я не получила в выводе

момент готов
спойлерТак готов
Возможность полезна
подробность свойственна
Книга шикарна
натура эгоистическая
пищи полна
желание неистовое
времени актуально
Сюжет интересный
не скучные
Юмор понятен
Рассказ небольшой
Книга знакома
школы знакома
оболочка актуальна
Тема актуальна
История неоднозначна
нить известна
чертой отличительной
Книга великолепна
писатель уникален
сэр зависим
герой зависим
друг зависим
товарища зависим
не похоже
не логична
обложке логична
Финал показался
облик важен
Не согласна
лицемерами красив
выбора красив
дрянь Мразь
человек циник
произведение актуально
время актуально
не известно
Язык остр
век обязан
человек обязан
Степень велика
соблазн велик
Сюжет интересен
Сюжет известен
произведение актуально
времена актуально
портрет старел
Смысл прост
не сопоставимы
Сюжет известен
персонажи известен
Книга многогранна
развитие логично
автор специфичный
конец хорош


Вывод не убираю, чтобы можно было увидеть оценки. Они _все_положительные. В конструкции с (кратким) прилагательным-предикатом с отрицанием "не" звучит хуже. "Книга интересна" звучит ОК, а "книга не интересна" звучит хуже, чем "книга неинтересная". <br>
Получается, достав именно такие оценки и проитерировавшись по схожим конструкциям в тестовой выборке можно было бы делать верные выводы

2. Снова ищем <b>эмфазу</b>, что поражает авторов рецензий (и в этом они выражают свое мнение). Тип: самый + прилагательное.

In [405]:
# 2

for sent in good_sents:
    occurence = 0
    for item in sent.tokens:
        if item.rel=='amod' and item.pos=='ADJ':
            occurence = 1
            head_id = item.id
            head_text= item.text
            
    if occurence ==1:
        for item in sent.tokens:
            if item.head_id == head_id and item.text.lower().startswith('сам'):
                print(item.text, head_text)
                print(sent.text)  # печатаю, чтобы проверить гипотезу

самым невероятным
Его заветное желание исполняется самым невероятным способом – портрет забирает на себя все пороки, грехи своего обладателя и его морщина.
самый высший
Безусловно, только самый высший балл.
самое прекрасное
Но не забывайте, что самое прекрасное в этом мире – это ВАШЕ лицо и ВАША уникальность!
Самое большое
Самое большое влияние над ним возымел лорд Генри.
самые необычные
Совращающая истина этой книги покоряет тебя с первых страниц, происходящее вызывает самые необычные эмоции.
Самый яркий
Самый яркий персонаж книги, как по мне.
самое главное
И самое главное произведение актуально и в наше время тоже.
самое вероятное
Ладно, может быть, логично, что смерть владельца такого портрета – самое вероятное развитие событий.


3. Ищу модификации прилагательных, опять же, какие качества считают настолько важными по смыслу, что это лексически выделяют через наречия. <br>
<b>наречие/частица + прилагательное</b><br><br>
Потом можно все такие "вызывающие эмоции" прилагательные закинуть в отдельный список, считать маркерами "положительной оценки", заранее исключив из списка "не".

In [407]:
#3
for sent in good_sents:
    occurence = 0
    for item in sent.tokens:
        if item.rel=='amod' and item.pos=='ADJ':
            occurence = 1
            head_id = item.id
            head_text= item.text 
       
    if occurence ==1:
        for item in sent.tokens:
            if item.head_id == head_id and (item.rel=='advmod'):
                print(item.text, head_text)

столь удивительную
еще долгое
очень долгое
же первых
Очень крутая
Очень интересный
очень актуальные
только высший
более исступленное
абсолютно неиспорченный
вечно молодом
очень приятный
более серьёзном
совсем небольшая
менее циническое
даже неоспоримые
не детская
лишь милого
далеко полный
не полный
более опытного
более известный
лучше кажущийся
почти чувственное
совершенно обычном
сильно материальным
необыкновенно красивым
оглушительно красивый
снова неприкрытая
довольно неприкрытая
хоть тысячную
лишь положительные
Вот единственный
очень нужное
слишком предсказуемое
просто потрясающую
безумно прекрасный
только детский
и небольшого


###### Небольшое заключение

1. Я не попробовала проверить на "негативном материале", но из рецензий с оценкой 5 материал вышел действительно хороший. Вычленяются действительно маркеры хорошей оценки.

\+ Я попробовала еще некоторые комбинации, но они хорошего выводы на давали) Поэтому те три кажутся самыми работающими для моего материала. Допустим, "не" + глагол мало что хорошее выдает. И advmod + глагол тоже ничего важного не дало)